# 0. Imports

In [37]:
%load_ext autoreload
%autoreload 2

# imports
import pandas as pd
import numpy as np
import datetime as dt
import time
import os
import pickle

from colorama import Fore, Style
from sklearn.pipeline import Pipeline
from joblib import dump, load

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Importing those functions from the API package so they can be used
# by the loaded pipeline in the API package later
from decipherer.ml_logic.encoders import ffill_nan, add_datetime_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
# environment variables
LOCAL_REGISTRY_PATH = "../training_outputs"

# prerequisite: having a "training_outputs" folder at the root of the project
! tree ../training_outputs

../training_outputs
├── metrics
│   ├── 20221123-163916.pickle
│   ├── 20221123-170451.pickle
│   ├── 20221124-115822.pickle
│   ├── 20221125-124411.pickle
│   ├── 20221125-141247.pickle
│   ├── 20221125-141615.pickle
│   └── 20221125-143102.pickle
├── models
│   ├── 20221123-163916.joblib
│   ├── 20221123-170451.joblib
│   ├── 20221124-115822.joblib
│   ├── 20221125-124411.joblib
│   ├── 20221125-141247.joblib
│   ├── 20221125-141615.joblib
│   └── 20221125-143102.joblib
└── params
    ├── 20221123-163916.pickle
    ├── 20221123-170451.pickle
    ├── 20221124-115822.pickle
    ├── 20221125-124411.pickle
    ├── 20221125-141247.pickle
    ├── 20221125-141615.pickle
    └── 20221125-143102.pickle

3 directories, 21 files


# 1. Build pipeline

In [39]:
# Get rid of nan values using ffil
# def ffill_nan(X):
#     return X.fillna(method='ffill', axis = 0)

In [40]:
# # Create new datetime features
# def add_datetime_features(X):
    
#     # Copy X to avoid pandas warning
#     X_rep = X.copy()
    
#     # Handle datetime format
#     datetime = pd.to_datetime(X_rep['date'] + ' ' + X_rep['time'], format="%d/%m/%Y %H:%M:%S")
    
#     # Create new features using month, weekday, hour and minute
#     X_rep['month'] = datetime.dt.month
#     X_rep['weekday'] = datetime.dt.weekday
#     X_rep['hour'] = datetime.dt.hour
#     X_rep['minute'] = datetime.dt.minute
    
#     # Consider periodic effects
#     X_rep['month_sin'] = np.sin(2*np.pi*X_rep['month']/12)
#     X_rep['month_cos'] = np.cos(2*np.pi*X_rep['month']/12)
    
#     # Get rid of Datetime
#     return X_rep.drop(columns=['date', 'time'])

In [41]:
# Create a pipeline to preprocess the data

n_estimators = 10

features = ['global_active_power', 'global_reactive_power', 'voltage', 'global_intensity', 'global_consumption']
datetimes = ['date', 'time']

preparator = ColumnTransformer([ 
    ('imputer', FunctionTransformer(ffill_nan), features),
    ('datetime_features_adder', FunctionTransformer(add_datetime_features), datetimes)
])

pipeline = Pipeline([
    ('preparator', preparator),
    ('std_scaler', StandardScaler()),
    ('estimator', RandomForestRegressor(n_estimators=n_estimators))
])

pipeline

Pipeline(steps=[('preparator',
                 ColumnTransformer(transformers=[('imputer',
                                                  FunctionTransformer(func=<function ffill_nan at 0x7f4e2df7d4c0>),
                                                  ['global_active_power',
                                                   'global_reactive_power',
                                                   'voltage',
                                                   'global_intensity',
                                                   'global_consumption']),
                                                 ('datetime_features_adder',
                                                  FunctionTransformer(func=<function add_datetime_features at 0x7f4e2df7da60>),
                                                  ['date', 'time'])])),
                ('std_scaler', StandardScaler()),
                ('estimator', RandomForestRegressor(n_estimators=10))])

# 3. Prepare the data to train

In [42]:
# Take just a subset of the data for now (year==2008)
data = pd.read_csv('../data/household_power_consumption.txt', sep=';', na_values='?')
data = data[data.Date.str.endswith('2008')]
data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 527040 entries, 547596 to 1074635
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date                   527040 non-null  object 
 1   time                   527040 non-null  object 
 2   global_active_power    526905 non-null  float64
 3   global_reactive_power  526905 non-null  float64
 4   voltage                526905 non-null  float64
 5   global_intensity       526905 non-null  float64
 6   sub_metering_1         526905 non-null  float64
 7   sub_metering_2         526905 non-null  float64
 8   sub_metering_3         526905 non-null  float64
dtypes: float64(7), object(2)
memory usage: 40.2+ MB


In [43]:
labels = ['sub_metering_1', 'sub_metering_2', 'sub_metering_3']
data['global_consumption'] = data[labels].sum(axis=1)

In [44]:
# Create X, y and save datetime in a separeted column
X = data.drop(columns=labels)
y = data[labels]
X.shape, y.shape

((527040, 7), (527040, 3))

In [45]:
y.isna().sum()

sub_metering_1    135
sub_metering_2    135
sub_metering_3    135
dtype: int64

In [46]:
# Remove na values from y
y = y.fillna(method='ffill', axis=0)

In [47]:
X.head()

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
547596,1/1/2008,00:00:00,1.620,0.070,241.25,6.6,18.0
547597,1/1/2008,00:01:00,1.626,0.072,241.74,6.6,18.0
547598,1/1/2008,00:02:00,1.622,0.072,241.52,6.6,18.0
547599,1/1/2008,00:03:00,1.612,0.070,240.82,6.6,18.0
547600,1/1/2008,00:04:00,1.612,0.070,240.80,6.6,18.0


In [48]:
y.head()

,sub_metering_1,sub_metering_2,sub_metering_3
547596,0.0,0.0,18.0
547597,0.0,0.0,18.0
547598,0.0,0.0,18.0
547599,0.0,0.0,18.0
547600,0.0,0.0,18.0


In [49]:
# Train/test Split /!\ For later, if we use sequential models (ARIMA, RNN): see if we consider a TS special train/test split (to keep sequence's logic)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((368928, 7), (158112, 7), (368928, 3), (158112, 3))

In [50]:
X_train.head()

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
782731,12/6/2008,06:55:00,1.494,0.098,238.55,6.2,20.0
846652,26/7/2008,16:16:00,1.564,0.218,239.58,6.6,23.0
831230,15/7/2008,23:14:00,0.666,0.098,243.53,2.6,1.0
1032275,2/12/2008,13:59:00,2.112,0.060,241.36,9.6,18.0
1012582,18/11/2008,21:46:00,2.468,0.218,237.71,10.6,9.0


# 4. Train and save the pipeline

In [51]:
# Function to save the pipeline locally
def save_pipeline(pipeline: Pipeline = None,
                  params: dict = None,
                  metrics: dict = None) -> None:
    """
    persist trained pipeline, params and metrics
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")

    print(Fore.BLUE + "\nSave pipeline to local disk..." + Style.RESET_ALL)

    # save params
    if params is not None:
        params_path = os.path.join(LOCAL_REGISTRY_PATH, "params", timestamp + ".pickle")
        with open(params_path, "wb") as file:
            pickle.dump(params, file)

    # save metrics
    if metrics is not None:
        metrics_path = os.path.join(LOCAL_REGISTRY_PATH, "metrics", timestamp + ".pickle")
        with open(metrics_path, "wb") as file:
            pickle.dump(metrics, file)

    # save pipeline
    if pipeline is not None:
        pipeline_path = os.path.join(LOCAL_REGISTRY_PATH, "models", timestamp + ".joblib")
        dump(pipeline, pipeline_path)
         
    print("\n✅ data saved locally")

    return None

In [52]:
%%time

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Mesure its performance
r2_score = pipeline.score(X_test, y_test)
#r2_score = pipeline.score(X_test[['Voltage']].dropna(), y_test.dropna())

# Save it locally
params = dict(
    # Model parameters
    n_estimators=10,

    # Package behavior
    context="train",

    # Data used to fit
    dataset_start=data.date.iloc[0],
    dataset_end=data.date.iloc[-1]
)

metrics = dict(r2_score=r2_score)
print(metrics)

save_pipeline(pipeline, params, metrics)

{'r2_score': 0.8976990494764903}

Save pipeline to local disk...

✅ data saved locally
CPU times: user 29.5 s, sys: 62 ms, total: 29.6 s
Wall time: 29.6 s


# 5. Load the pipeline and try to predict

In [53]:
# To load a specific local pipeline
# pipeline_path = os.path.join(LOCAL_REGISTRY_PATH, "models", "20221125-143102.joblib")
# pipeline_loaded = load(pipeline_path)
# pipeline_loaded

Pipeline(steps=[('preparator',
                 ColumnTransformer(transformers=[('imputer',
                                                  FunctionTransformer(func=<function ffill_nan at 0x7f4e2df7d4c0>),
                                                  ['global_active_power',
                                                   'global_reactive_power',
                                                   'voltage',
                                                   'global_intensity',
                                                   'global_consumption']),
                                                 ('datetime_features_adder',
                                                  FunctionTransformer(func=<function add_datetime_features at 0x7f4e2df7da60>),
                                                  ['date', 'time'])])),
                ('std_scaler', StandardScaler()),
                ('estimator', RandomForestRegressor(n_estimators=10))])

In [54]:
# X_test

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
677894,31/3/2008,11:38:00,1.408,0.102,242.69,5.8,18.0
756536,25/5/2008,02:20:00,0.640,0.228,240.71,2.8,1.0
936361,26/9/2008,23:25:00,1.370,0.206,239.05,5.8,18.0
924449,18/9/2008,16:53:00,0.288,0.112,240.57,1.2,1.0
817945,6/7/2008,17:49:00,3.176,0.518,238.72,13.4,41.0
...,...,...,...,...,...,...,...
955508,10/10/2008,06:32:00,0.334,0.110,239.73,1.4,2.0
979845,27/10/2008,04:09:00,0.210,0.050,243.70,0.8,0.0
956868,11/10/2008,05:12:00,1.476,0.200,240.52,6.2,18.0
946325,3/10/2008,21:29:00,3.960,0.218,236.25,16.8,38.0


In [55]:
# y_pred = pd.DataFrame(pipeline_loaded.predict(X_test), columns=labels)
# y_pred['datetime'] = pd.to_datetime(X_test['date'] + ' ' + X_test['time'], format="%d/%m/%Y %H:%M:%S")
# y_pred

,sub_metering_1,sub_metering_2,sub_metering_3,datetime
0,0.0,0.0,18.0,NaT
1,0.0,0.0,1.0,NaT
2,0.0,0.0,18.0,NaT
3,0.0,0.0,1.0,NaT
4,31.4,4.8,3.7,NaT
...,...,...,...,...
158107,0.0,1.3,0.7,NaT
158108,0.0,0.0,0.0,NaT
158109,0.0,0.0,18.0,NaT
158110,38.0,0.0,0.0,NaT
